In [ ]:
import os
import h5py
import json
import pandas as pd
import numpy as np
import torch
import random
import shutil
import rmm
import cupy as cp

from datetime import datetime
from preprocess.loadConfig import ConfigLoader
from preprocess.loadLPdata import LoadLPData
from preprocess.reconstruct3D import process_groups_reconstruction
from preprocess.egocentric import egocentric_alignment_3d, align_3dvector_to_axis
from preprocess.computemotion import compute_vector_magnitudes, compute_velocities,compute_direction_changes
from preprocess.addposition import calculate_position
from preprocess.addangle import calculate_normal_vectors_per_group, calculate_angle_between_line_and_plane
from analysis.dim_reduction.pca import perform_pca, save_pca_results
from analysis.cwt import CWT, cwt_filter
from analysis.dim_reduction.umap import perform_umap, plot_umap
from analysis.embedding.vae.vae_main import vae_run
from analysis.embedding.vae.vae_visualize import extract_latent_space
from analysis.embedding.vae.vae_utils import numpy_to_tensor, load_savedmodel, load_hyperparameters
from analysis.segmentation.hdbscan.hdbscan_clustering import apply_clustering
from analysis.segmentation.hdbscan.hdbscan_confirm import load_video_frames, plot_frames, extract_cluster_frame_indices
from analysis.segmentation.hdbscan.hdbscan_medoid import calculate_cluster_medoids, plot_latent_space_with_medoids,plot_latent_space_with_medoids2

In [ ]:
########## Pose tracking dataset of Sleap-Anipose #########
h5_files = [
"sleap-anipose result file pathways(.h5), that 3d coordinates time series dataset"
]

In [ ]:
##########  Configuration ######################################################
print("\n***Reading the configuration file***")
config_path = 'file pathway/to/ .. /config_slp_sample.json'
cfg = ConfigLoader(config_path)

# Load configuration in each process
info_cfg = cfg.get_info()
cwt_cfg = cfg.get_cwt()
vae_cfg = cfg.get_vae()
umap_cfg = cfg.get_umap()
hdbscan_cfg = cfg.get_hdbscan()

In [ ]:
########## Create the index_info.csv file #########
rows_per_file = #the number of frames in each video
sourcefile_column = [file for file in h5_files for _ in range(rows_per_file)]

# create dataframe
df = pd.DataFrame({"Sourcefile": sourcefile_column})

# save into csv file
df.to_csv("pathway/../index_info.csv", index=False)

print("Create the index_info.csv file!")

In [ ]:
########## LOAD multiple h5 files
all_data = []
all_sources = []  # list of saving index 

for idx, file in enumerate(h5_files):
    with h5py.File(file, "r") as f:
        locations = f["tracks"][:].squeeze()  # (n_frames, n_nodes, 3)
        relocations = locations.reshape(locations.shape[0], -1)  # (n_frames, n_nodes*3)

        output_dir = os.path.join(info_cfg["save_dir"], "npy") 
        os.makedirs(output_dir, exist_ok=True)

        npy_filename = os.path.join(output_dir, os.path.basename(file).replace(".h5", ".npy"))
        np.save(npy_filename, relocations)

        all_data.append(relocations)

        source_idx = np.full((relocations.shape[0], 1), idx) 
        all_sources.append(source_idx)

data = np.vstack(all_data)  # (total_frames, n_nodes*3)
np.save(os.path.join(output_dir, "total data.npy"), data)
np.save(os.path.join(output_dir, "source.npy"), np.vstack(all_sources))

In [ ]:
######## Preprocess ########################################################
#### Change the alignment: Allocentric to Egocentric
# Egocentric Alignment
ego_coords, center_coords = egocentric_alignment_3d(data, info_cfg["keypoint_names"], "body")
print("Convert 3Ddata into egocentric. Shape:", ego_coords.shape)
aligned_rotated_data = align_3dvector_to_axis(
    ego_coords,
    vector_from_name='body',  # same as egocentric_keypoint
    vector_to_name='top_tail',
    bodypoint_names=info_cfg["keypoint_names"]
)
print("Egocentric-Axis Alignment completed for 3D data. Shape:", aligned_rotated_data.shape)
denoised_data = aligned_rotated_data

#### Compute velocity: coords to velocities
arr_velocity = compute_velocities(ego_coords)
print("Completion of computing velocities. Shape:", arr_velocity.shape)

preprocess_path = os.path.join(info_cfg["save_dir"],"mouse_kaist_ego_axis_v.npy")
np.save(preprocess_path, arr_velocity)
print(f"Preprocess result saved to {preprocess_path}.")

denoised_data = arr_velocity

# ### Denoising the input of CWT
# _, _ , denoised_data,_, _ = perform_pca(arr_velocity, 0.95, using_gpu=False)
# print(f"After denoising, the cwt input data shape is :{denoised_data.shape}") 


In [ ]:
#### (Option) Concatenate posture and velocity BEFORE CWT ########
from sklearn.preprocessing import StandardScaler
scaler_coords = StandardScaler()
scaler_speed = StandardScaler()

coords_scaled = scaler_coords.fit_transform(aligned_rotated_data)
#coords_scaled = scaler_coords.fit_transform(ego_coords), choosing appropriate one that u want to use
speed_scaled = scaler_speed.fit_transform(arr_velocity)

denoised_data = np.concatenate([coords_scaled, speed_scaled], axis=1)
print(f"The CWT input data shape is : {denoised_data.shape}, concatenated posture and velocity")

In [ ]:
######## Continuous Wavelet Transform ##########################################
if cwt_cfg["use_saved_cwt"]:
    print(f"\n***Loaded precomputed CWT data from {cwt_cfg['use_savedcwt_path']}***")
    amp_reshaped = np.load(cwt_cfg["use_savedcwt_path"])
    print("Loaded CWT data. Shape.T:", amp_reshaped.T.shape)
else: 
    print("\n***Performing Continuous wavelet transform***")
    # Create an instance of the CWT class
    cwt = CWT(cwt_cfg["frequencies"], cwt_cfg["cwt_omega0"], cwt_cfg["cwt_dt"], cwt_cfg["scaler"])  

    # Perform the wavelet transform
    amp, W = cwt.fast_wavelet_morlet_convolution(denoised_data)

    # Save the cwt result 
    L, N, num_features = amp.shape # (50,17546,104)
    amp_reshaped = np.reshape(amp, (L*num_features, N)) # (50*104, 17546)

    cwt_dir = os.path.join(info_cfg["save_dir"], 'cwt')
    os.makedirs(cwt_dir, exist_ok=True)

    cwt_path = os.path.join(cwt_dir,f'cwt_{cwt_cfg["frequencies_start"]}_{cwt_cfg["frequencies_end"]}_{cwt_cfg["frequencies_step"]}_{cwt_cfg["scaler"]}.npy')
    np.save(cwt_path, amp_reshaped)
    print(f"CWT result saved to {cwt_path}. Shape.T:", amp_reshaped.T.shape)

    # Plot and save the CWT results
    if cwt_cfg["cwt_plot_separate"]:
        cwt.plot_cwt_separate(amp, save_path=cwt_dir)
        print("Saved CWT plots")
    amp_reshaped_before = amp_reshaped
    # Filtering and save the CWT results
    if cwt_cfg['cwt_filtering']:
        amp_reshaped, retained_freq = cwt_filter(amp,cwt_cfg['freq_removal_threshold'],cwt_dir)
        cwt_path = os.path.join(cwt_dir,f'cwt_{cwt_cfg["frequencies_start"]}_{cwt_cfg["frequencies_end"]}_{cwt_cfg["frequencies_step"]}_{cwt_cfg["scaler"]}_filter.npy')
        cwt_path2 = os.path.join(cwt_dir,f'cwt_{cwt_cfg["frequencies_start"]}_{cwt_cfg["frequencies_end"]}_{cwt_cfg["frequencies_step"]}_{cwt_cfg["scaler"]}_filterinfo.npy')
        np.save(cwt_path, amp_reshaped)
        np.savez(cwt_path2, *retained_freq)
        print(f"Complete filtering the CWT data, before VAE. saved to {cwt_path}")
        print('Shape.T:', amp_reshaped.T.shape)

# # Denoising before performing vae
# print("\n***Denoising the input of VAE")
# amp_reshaped,_,_,_,_ = perform_pca(amp_reshaped.T, confidence = preprocess_cfg["pca_confidence"])
# amp_reshaped = amp_reshaped.T

In [ ]:
#### (Option) Concatenate posture and velocity AFTER CWT ######## (n_features, n_samples)
cwt_posture = np.load('path/to/../cwt_posture.npy')
cwt_velocity = np.load('path/to/../cwt_posture.npy')

# Scaling seperately BEFORE VAE and Save the result
from sklearn.preprocessing import StandardScaler
scaler_posture  = StandardScaler()
scaler_velocity = StandardScaler()

cwt_scaled_posture = scaler_posture.fit_transform(cwt_posture.T)
cwt_scaled_velocity = scaler_velocity.fit_transform(cwt_velocity.T)
print("Finish scaling seperately before VAE")

cwt_concat = np.concatenate([cwt_scaled_posture, cwt_scaled_velocity], axis=1) # (n_samples, n_features)
print(f"The VAE input data shape.T is : {cwt_concat.shape}, concatenated cwt results of posture and velocity.")
cwt_concat_dir = os.path.join(info_cfg["save_dir"], 'cwt')
os.makedirs(cwt_concat_dir, exist_ok=True)
np.save(os.path.join(cwt_concat_dir,'cwt_scaled_p_v.npy'), cwt_concat.T) # save into (n_features, n_samples) format

amp_reshaped = cwt_concat.T

In [ ]:
#### (Option; Restart from vae) Load CWT RESULT(cwt_concat: cwt_scaled_posture + cwt_scaled_velocity) ################
amp_reshaped = np.load('/home/jiyoung/Desktop/lai_240524/Post-pose3D/test_kaist/kaist_data/debug14_p_v_1/cwt/cwt_scaled_p_v.npy')
print(f"The VAE input data shape.T is : {amp_reshaped.T.shape}, concatenated cwt results of posture and velocity.")

In [ ]:
######## Embedding: Variational AutoEncoder ##################################
if not umap_cfg["use_saved_umap"]:   
    if vae_cfg["use_saved_latentspace"]:
        print(f"\n***Loaded VAE latentspace data from {vae_cfg['use_savedlatent_path']}***")
        latent_space = np.load(vae_cfg['use_savedlatent_path'])
        print("Loaded VAE latent space. Shape:",latent_space.shape)
    else:
        if vae_cfg["use_saved_vae"]:
            print("\n***Using already existed vae_model***")
            amp_reshaped = numpy_to_tensor(amp_reshaped) # (17546,50*104) = (n_samples*n_features)
            vae_trained_model, device = load_savedmodel(vae_cfg["vaemodel_type"],vae_cfg["use_saved_vaemodel_path"],
                                                        vae_cfg["use_saved_vaehyparam_path"], amp_reshaped.shape[1])
        else:
            print("\n***Start training of vae_model***") 
            vae_trained_model, device = vae_run(config_path, amp_reshaped)
            # vae_trained_model = vae_run(config_path) # vae_cfg["data_path"]에 원하는 Data(.npy) 경로 넣으면 해당 데이터로 vae run
            amp_reshaped = numpy_to_tensor(amp_reshaped)

        # Extract latent space from VAE model and save it
        latent_space = extract_latent_space(vae_trained_model, amp_reshaped, device, batch_size=random.choice(vae_cfg["batch_size_options"])) #(n_samples*n_features)
        np.save(os.path.join(info_cfg["save_dir"],'vae_latentspace.npy'),latent_space)
        del vae_trained_model, amp_reshaped

In [ ]:
######## Dim Reduction: UMAP ################################################
if umap_cfg["use_saved_umap"]:
    print(f"\n***Loaded precomputed UMAP data from {umap_cfg['use_savedumap_path']}***")
    umap_space = np.load(umap_cfg["use_savedumap_path"])
else: 
    umap_space, umap_save_dir = perform_umap(latent_space, umap_cfg["n_neighbors"],
                                    umap_cfg["min_dist"],umap_cfg["n_components"],
                                    info_cfg["save_dir"])
    plot_umap(umap_space, umap_cfg['n_neighbors'], umap_cfg['min_dist'], umap_cfg['plot_2d'],
            umap_cfg['plot_3d'], umap_cfg['save_plot'], umap_save_dir)
    del latent_space
torch.cuda.empty_cache()

In [ ]:
# ### if we need deterministic results, use cpu version!!!
# import umap as umap_cpu
# umap_model = umap_cpu.UMAP(
#             n_neighbors= umap_cfg["n_neighbors"],
#             min_dist=umap_cfg["min_dist"],
#             n_components=umap_cfg["n_components"],
#             random_state=42
#         )
# umap_space_cpu = umap_model.fit_transform(latent_space)

In [ ]:
######## Clustering: HDBSCAN ################################################
min_cluster_sizes = hdbscan_cfg["min_cluster_sizes"]
for min_cluster_size in min_cluster_sizes:
    print(f"\n***Running HDBSCAN with min_cluster_size = {min_cluster_size}***")

    # HDBSCAN
    try:
        reduced_data, cluster_labels, soft_labels, max_probs, index_info_df = apply_clustering(
             umap_space, hdbscan_cfg, min_cluster_size, info_cfg["save_dir"])
        cluster_save_dir= os.path.join(info_cfg["save_dir"],'hdbscan/pdf')
        os.makedirs(cluster_save_dir, exist_ok=True)
        #np.save(os.path.join(cluster_save_dir,'reduced_data.npy'),reduced_data)
        np.save(os.path.join(cluster_save_dir,f'soft_labels_{hdbscan_cfg["method"]}_{min_cluster_size}.npy'),soft_labels)
        # reduced_data, cluster_labels, soft_labels, max_probs, index_info_df = apply_clustering(
        #     latent_space, hdbscan_cfg, min_cluster_size, info_cfg["save_dir"])
        print(">>> Clustering completed successfully.")
    except Exception as e:
        print(f">>> HDBSCAN clustering failed: {e}")

    # GPU Memory Cleanup
    cp.get_default_memory_pool().free_all_blocks()